In [10]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

chrome_version = "142.0.7444.176"

service = Service(
    ChromeDriverManager(driver_version=chrome_version).install()
)
driver = webdriver.Chrome(service=service)


driver.maximize_window()

wait = WebDriverWait(driver, 5)

def wait_for_page_to_load(driver, wait):

    page_title = driver.title
    
    try:
        
        wait.until(
            lambda driver : driver.execute_script("return document.readyState") == 'complete'
        )
    
    except Exception as e:
        print(f"page {page_title} was not loaded within time with error msg : {e}")
    
    else:
        print(f"page {page_title} was loaded within time")


url = "https://finance.yahoo.com/"
driver.get(url)

wait_for_page_to_load(driver, wait)

actions = ActionChains(driver)

market_menu = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="navigation-container"]/ol/li[3]/a/div'))
)

actions.move_to_element(market_menu).perform()


stocks_menu = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="navigation-container"]/ol/li[3]/ol/li[1]/a/span'))
)

actions.move_to_element(stocks_menu).perform()

trending_menu = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="navigation-container"]/ol/li[3]/ol/li[1]/ol/li[4]/a/span'))
)

trending_menu.click()

wait_for_page_to_load(driver, wait)

most_active = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="tab-most-active"]'))
)

most_active.click()

wait_for_page_to_load(driver, wait)

data = []


while True:

    wait.until(
        EC.presence_of_element_located((By.TAG_NAME, 'table'))
    )

    rows = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')
    for row in rows:
        values = row.find_elements(By.TAG_NAME, 'td')
        print([val.text for val in values])
        stock = {
            "name": values[1].text,
            "symbol": values[0].text,
            "price": values[3].text,
            "change": values[4].text,
            "volumn": values[6].text,
            "market_cap": values[8].text,
            "pe_ratio": values[9].text
        }
        data.append(stock)
    
    try:
        
        next_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content-wrapper"]/section[1]/div/div[4]/div[3]/button[3]'))
        )

    except:
        print("the \"next\" button is not clickable as we have gone through all pages.")
        break
    else:
        next_button.click()
        
        time.sleep(1)
    



driver.quit()

page Yahoo Finance - Stock Market Live, Quotes, Business & Finance News was loaded within time
page Yahoo Finance - Stock Market Live, Quotes, Business & Finance News was loaded within time
page Top Trending Stocks: US stocks with the highest interest today - Yahoo Finance was loaded within time
['OLMA', 'Olema Pharmaceuticals, Inc.', '', '20.14', '+11.62', '+136.38%', '85.927M', '1.194M', '1.383B', '--', '-2.29%', '2.86\n27.17']
['HD', 'The Home Depot, Inc.', '', '336.48', '-21.55', '-6.02%', '9.079M', '3.458M', '334.928B', '22.86', '-11.99%', '326.31\n439.37']
['WBD', 'Warner Bros. Discovery, Inc.', '', '23.69', '+0.95', '+4.18%', '58.421M', '51.304M', '58.703B', '124.68', '+139.37%', '7.52\n24.19']
['MSFT', 'Microsoft Corporation', '', '493.79', '-13.70', '-2.70%', '32.649M', '21.163M', '3.67T', '35.12', '+21.47%', '344.79\n555.45']
['NET', 'Cloudflare, Inc.', '', '196.53', '-5.72', '-2.83%', '8.033M', '2.505M', '68.841B', '--', '+109.87%', '89.42\n260.00']
['TMC', 'TMC the metals c

In [11]:
len(data)

315

In [12]:
stocks_df = pd.DataFrame(data)
stocks_df

,name,symbol,price,change,volumn,market_cap,pe_ratio
0,"Olema Pharmaceuticals, Inc.",OLMA,20.14,+11.62,85.927M,1.383B,--
1,"The Home Depot, Inc.",HD,336.48,-21.55,9.079M,334.928B,22.86
2,"Warner Bros. Discovery, Inc.",WBD,23.69,+0.95,58.421M,58.703B,124.68
3,Microsoft Corporation,MSFT,493.79,-13.70,32.649M,3.67T,35.12
4,"Cloudflare, Inc.",NET,196.53,-5.72,8.033M,68.841B,--
...,...,...,...,...,...,...,...
310,Southwest Airlines Co.,LUV,31.96,+0.11,5.059M,16.785B,49.35
311,British American Tobacco p.l.c.,BTI,54.86,+0.15,5.058M,119.367B,29.58
312,"Asana, Inc.",ASAN,12.09,+0.06,5.046M,2.856B,--
313,"General Mills, Inc.",GIS,47.69,+0.72,5.036M,25.499B,9.00


In [13]:
stocks_df.info

<bound method DataFrame.info of                                 name symbol   price  change   volumn  \
0        Olema Pharmaceuticals, Inc.   OLMA   20.14  +11.62  85.927M   
1               The Home Depot, Inc.     HD  336.48  -21.55   9.079M   
2       Warner Bros. Discovery, Inc.    WBD   23.69   +0.95  58.421M   
3              Microsoft Corporation   MSFT  493.79  -13.70  32.649M   
4                   Cloudflare, Inc.    NET  196.53   -5.72   8.033M   
..                               ...    ...     ...     ...      ...   
310           Southwest Airlines Co.    LUV   31.96   +0.11   5.059M   
311  British American Tobacco p.l.c.    BTI   54.86   +0.15   5.058M   
312                      Asana, Inc.   ASAN   12.09   +0.06   5.046M   
313              General Mills, Inc.    GIS   47.69   +0.72   5.036M   
314         Palo Alto Networks, Inc.   PANW  201.00   -1.90   5.001M   

    market_cap pe_ratio  
0       1.383B       --  
1     334.928B    22.86  
2      58.703B   124.68  

In [14]:
stocks_df.describe

<bound method NDFrame.describe of                                 name symbol   price  change   volumn  \
0        Olema Pharmaceuticals, Inc.   OLMA   20.14  +11.62  85.927M   
1               The Home Depot, Inc.     HD  336.48  -21.55   9.079M   
2       Warner Bros. Discovery, Inc.    WBD   23.69   +0.95  58.421M   
3              Microsoft Corporation   MSFT  493.79  -13.70  32.649M   
4                   Cloudflare, Inc.    NET  196.53   -5.72   8.033M   
..                               ...    ...     ...     ...      ...   
310           Southwest Airlines Co.    LUV   31.96   +0.11   5.059M   
311  British American Tobacco p.l.c.    BTI   54.86   +0.15   5.058M   
312                      Asana, Inc.   ASAN   12.09   +0.06   5.046M   
313              General Mills, Inc.    GIS   47.69   +0.72   5.036M   
314         Palo Alto Networks, Inc.   PANW  201.00   -1.90   5.001M   

    market_cap pe_ratio  
0       1.383B       --  
1     334.928B    22.86  
2      58.703B   124.68

In [15]:
stocks_df.apply(lambda col : col.str.strip() if col.dtype == 'object' else col)

,name,symbol,price,change,volumn,market_cap,pe_ratio
0,"Olema Pharmaceuticals, Inc.",OLMA,20.14,+11.62,85.927M,1.383B,--
1,"The Home Depot, Inc.",HD,336.48,-21.55,9.079M,334.928B,22.86
2,"Warner Bros. Discovery, Inc.",WBD,23.69,+0.95,58.421M,58.703B,124.68
3,Microsoft Corporation,MSFT,493.79,-13.70,32.649M,3.67T,35.12
4,"Cloudflare, Inc.",NET,196.53,-5.72,8.033M,68.841B,--
...,...,...,...,...,...,...,...
310,Southwest Airlines Co.,LUV,31.96,+0.11,5.059M,16.785B,49.35
311,British American Tobacco p.l.c.,BTI,54.86,+0.15,5.058M,119.367B,29.58
312,"Asana, Inc.",ASAN,12.09,+0.06,5.046M,2.856B,--
313,"General Mills, Inc.",GIS,47.69,+0.72,5.036M,25.499B,9.00


In [90]:
stocks_df.dtypes

name          object
symbol        object
price         object
change        object
volumn        object
market_cap    object
pe_ratio      object
dtype: object

In [16]:
stocks_df['price']= stocks_df['price'].astype(float)

In [17]:
stocks_df.dtypes

name           object
symbol         object
price         float64
change         object
volumn         object
market_cap     object
pe_ratio       object
dtype: object

In [18]:
import re

In [19]:
stocks_df['change'].str.extract(r'([^0-9.])')

,0
0,+
1,-
2,+
3,-
4,-
...,...
310,+
311,+
312,+
313,+


In [20]:
stocks_df['change'] = stocks_df['change'].str.replace('+','')

In [21]:
stocks_df

,name,symbol,price,change,volumn,market_cap,pe_ratio
0,"Olema Pharmaceuticals, Inc.",OLMA,20.14,11.62,85.927M,1.383B,--
1,"The Home Depot, Inc.",HD,336.48,-21.55,9.079M,334.928B,22.86
2,"Warner Bros. Discovery, Inc.",WBD,23.69,0.95,58.421M,58.703B,124.68
3,Microsoft Corporation,MSFT,493.79,-13.70,32.649M,3.67T,35.12
4,"Cloudflare, Inc.",NET,196.53,-5.72,8.033M,68.841B,--
...,...,...,...,...,...,...,...
310,Southwest Airlines Co.,LUV,31.96,0.11,5.059M,16.785B,49.35
311,British American Tobacco p.l.c.,BTI,54.86,0.15,5.058M,119.367B,29.58
312,"Asana, Inc.",ASAN,12.09,0.06,5.046M,2.856B,--
313,"General Mills, Inc.",GIS,47.69,0.72,5.036M,25.499B,9.00


In [22]:
stocks_df.dtypes

name           object
symbol         object
price         float64
change         object
volumn         object
market_cap     object
pe_ratio       object
dtype: object

In [23]:
stocks_df['change'] = stocks_df['change'].astype(float)

In [24]:
stocks_df.dtypes

name           object
symbol         object
price         float64
change        float64
volumn         object
market_cap     object
pe_ratio       object
dtype: object

In [25]:
stocks_df['volumn'].unique()

array(['85.927M', '9.079M', '58.421M', '32.649M', '8.033M', '75.542M',
       '65.744M', '64.447M', '64.003M', '58.147M', '56.968M', '55.466M',
       '54.705M', '52.965M', '52.813M', '52.59M', '52.536M', '52.025M',
       '50.583M', '49.063M', '46.565M', '46.056M', '45.301M', '43.511M',
       '42.306M', '41.898M', '40.994M', '40.078M', '39.673M', '39.623M',
       '38.855M', '38.721M', '37.811M', '35.705M', '35.114M', '35.082M',
       '34.722M', '33.827M', '33.132M', '33.087M', '32.342M', '32.293M',
       '31.801M', '31.755M', '31.73M', '28.739M', '28.635M', '28.394M',
       '28.371M', '27.556M', '27.399M', '26.966M', '26.628M', '26.435M',
       '26.306M', '25.98M', '25.598M', '25.193M', '24.903M', '24.857M',
       '24.804M', '24.541M', '24.518M', '23.924M', '23.798M', '23.749M',
       '23.72M', '23.453M', '23.29M', '22.874M', '22.683M', '22.255M',
       '21.252M', '20.647M', '20.565M', '20.29M', '20.244M', '20.095M',
       '20.073M', '19.606M', '19.1M', '18.996M', '18.501M',

In [26]:
stocks_df['volumn'] = stocks_df['volumn'].apply(lambda x : x.replace('M',''))

In [27]:
stocks_df['volumn']

0      85.927
1       9.079
2      58.421
3      32.649
4       8.033
        ...  
310     5.059
311     5.058
312     5.046
313     5.036
314     5.001
Name: volumn, Length: 315, dtype: object

In [28]:
stocks_df.dtypes

name           object
symbol         object
price         float64
change        float64
volumn         object
market_cap     object
pe_ratio       object
dtype: object

In [29]:
stocks_df['volumn'] = stocks_df['volumn'].astype(float)

In [30]:
stocks_df.dtypes

name           object
symbol         object
price         float64
change        float64
volumn        float64
market_cap     object
pe_ratio       object
dtype: object

In [31]:
stocks_df['market_cap']= stocks_df['market_cap'].apply(
    lambda x : float(x.replace('B','')) if 'B' in x else float(x.replace('T',''))*1000 
)

In [32]:
stocks_df['market_cap'].str.extract(r'([^0-9.])', expand=False).unique()

AttributeError: Can only use .str accessor with string values!

In [33]:
stocks_df['market_cap']

0         1.383
1       334.928
2        58.703
3      3670.000
4        68.841
         ...   
310      16.785
311     119.367
312       2.856
313      25.499
314     137.481
Name: market_cap, Length: 315, dtype: float64

In [34]:
stocks_df.dtypes

name           object
symbol         object
price         float64
change        float64
volumn        float64
market_cap    float64
pe_ratio       object
dtype: object

In [35]:
stocks_df

,name,symbol,price,change,volumn,market_cap,pe_ratio
0,"Olema Pharmaceuticals, Inc.",OLMA,20.14,11.62,85.927,1.383,--
1,"The Home Depot, Inc.",HD,336.48,-21.55,9.079,334.928,22.86
2,"Warner Bros. Discovery, Inc.",WBD,23.69,0.95,58.421,58.703,124.68
3,Microsoft Corporation,MSFT,493.79,-13.70,32.649,3670.000,35.12
4,"Cloudflare, Inc.",NET,196.53,-5.72,8.033,68.841,--
...,...,...,...,...,...,...,...
310,Southwest Airlines Co.,LUV,31.96,0.11,5.059,16.785,49.35
311,British American Tobacco p.l.c.,BTI,54.86,0.15,5.058,119.367,29.58
312,"Asana, Inc.",ASAN,12.09,0.06,5.046,2.856,--
313,"General Mills, Inc.",GIS,47.69,0.72,5.036,25.499,9.00


In [44]:
stocks_df['pe_ratio'].str.extract(r'([^0-9.])', expand=False).unique()

array([nan], dtype=object)

In [39]:
stocks_df['pe_ratio'] = stocks_df['pe_ratio'].str.replace(',','')

In [43]:
stocks_df['pe_ratio'] = stocks_df['pe_ratio'].str.replace('--','')

In [40]:
stocks_df['pe_ratio'].unique()

array(['--', '22.86', '124.68', '35.12', '279.03', '11.28', '51.32',
       '14.56', '118.64', '33.16', '788.93', '1612.20', '5.64', '37.02',
       '15.53', '27.27', '46.48', '129.23', '13.42', '19.16', '26.51',
       '23.32', '14.37', '8.31', '22.28', '269.29', '32.52', '30.67',
       '36.52', '36.29', '9.67', '17.82', '26.63', '16.19', '51.24',
       '9.25', '169.64', '28.97', '68.88', '13.88', '12.30', '11.17',
       '11.00', '4.57', '28.09', '26.44', '188.56', '26.93', '14.42',
       '30.12', '8.76', '26.48', '7.31', '51.60', '14.06', '27.96',
       '24.52', '34.68', '13.82', '22.73', '12.71', '87.97', '219.23',
       '20.37', '9.93', '13.33', '12.62', '11.75', '21.96', '13.58',
       '15.70', '38.68', '20.07', '63.78', '74.31', '10.82', '21.51',
       '5.44', '380.22', '13.09', '22.47', '14.01', '16.17', '1697.31',
       '15.39', '22.67', '17.32', '17.68', '107.75', '23.59', '47.16',
       '9.78', '26.27', '12.60', '49.96', '9.24', '18.93', '24.73',
       '23.57', '76

In [41]:
stocks_df['pe_ratio'] = stocks_df['pe_ratio'].replace(',','')

In [42]:
stocks_df['pe_ratio'] = pd.to_numeric(stocks_df['pe_ratio'])

ValueError: Unable to parse string "--" at position 0

In [ ]:
stocks_df.dtypes

name             object
symbol           object
price_usd       float64
change          float64
volumn          float64
market_cap_B    float64
pe_ratio        float64
dtype: object

In [ ]:
stocks_df.rename(
   columns =  {
       "price": "price_usd",
		"volume": "volume_M",
		"market_cap": "market_cap_B"
       }, inplace=True
)

In [ ]:
stocks_df

,name,symbol,price_usd,change,volumn,market_cap_B,pe_ratio
0,"Tesla, Inc.",TSLA,445.91,-16.35,104.867,1483.000,303.34
1,Snap Inc.,SNAP,8.01,0.71,155.951,13.536,NaN
2,Pfizer Inc.,PFE,24.85,0.24,111.564,195.108,14.12
3,"Tesla, Inc.",TSLA,445.88,-16.38,96.048,1483.000,306.57
4,Opendoor Technologies Inc.,OPEN,6.56,-0.67,89.793,4.868,NaN
...,...,...,...,...,...,...,...
352,Vodafone Group Public Limited Company,VOD,11.34,0.07,5.046,27.731,NaN
353,Fastenal Company,FAST,40.77,-0.91,5.043,46.805,38.63
354,"DigitalOcean Holdings, Inc.",DOCN,47.08,1.27,5.043,4.306,14.70
355,"Fluence Energy, Inc.",FLNC,19.82,-0.90,5.019,3.617,NaN


In [ ]:
stocks_df.isnull().sum()

name              0
symbol            0
price_usd         0
change            0
volumn            0
market_cap_B      0
pe_ratio        155
dtype: int64

In [ ]:
stocks_df.to_excel("yahoo-stocks-data.xlsx", index=False)